In [6]:
%matplotlib inline
# import all shogun classes
from shogun import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import time
import os
import pandas as pd
import collections
etfData = collections.defaultdict(dict)
import stylised_facts_data_utilities.createLOB as createLOB
import collections
import pickle
import array_to_latex as a2l
import tqdm
import time

In [2]:
import stylised_facts_data_utilities.mmd as mmd
import stylised_facts_data_utilities.ksd as ksd
import stylised_facts_data_utilities.longtaildistr as ltd


/home/ak/anaconda3/envs/ds3_kernels/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ak/anaconda3/envs/ds3_kernels/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ak/anaconda3/envs/ds3_kernels/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ak/anaconda3/envs/ds3_kernels/lib/pyt

In [7]:
# getting all the data in 
ActivityCLockData= '/media/ak/My Passport2/Experiment Data/ActivityClockData/'
OHLCData= '/media/ak/My Passport2/Experiment Data/OHLCData/'
folderList = os.listdir(ActivityCLockData)
OHLCDataList = os.listdir(OHLCData)
ActivityClockDataList = os.listdir(ActivityCLockData)
figuresFolder = '/home/ak/Documents/Research/Papers/StylisedFactsPaper/figures/'
symbols =['FB1','JB1','FV1','G_1']
futuresData = collections.defaultdict(dict)

In [8]:

for symbol in tqdm.tqdm(symbols):
    print(symbol)
    for file in list(np.sort([s for s in os.listdir(ActivityCLockData) if 'Clocks' in s and (str(symbol)) in s])):
        pklClockFile = "".join((ActivityCLockData,file))
        #print(pklClockFile.split("_")[-2])
        futuresData[symbol][pklClockFile.split("_")[-2]]=pickle.load(open(pklClockFile , "rb"))['TickBarDf']
                                                                                                             
                                                                                                             
                                                                                                             
                               

  0%|          | 0/4 [00:00<?, ?it/s]

FB1


 25%|██▌       | 1/4 [00:02<00:07,  2.67s/it]

JB1


 50%|█████     | 2/4 [00:11<00:08,  4.46s/it]

FV1


 75%|███████▌  | 3/4 [03:40<01:05, 65.91s/it]

G_1


100%|██████████| 4/4 [04:28<00:00, 67.01s/it]


In [ ]:
futuresDataKeys = list(futuresData.keys())
symbolDates = list(futuresData[futuresDataKeys[0]].keys())

In [5]:
futuresDataKeys

NameError: name 'futuresDataKeys' is not defined

In [ ]:
[['BidSize', 'QuoteTime','BestBid',
       'TradeTime', 'AskSize','BestAsk', 'TradeVolume', 'TradedTime', 'type',
       'TradePrice', 'TimeStamp', 'milliSeconds','DollarVolume', 'MicroPrice', 'TradeSize', 'DollarVolumeTraded']]
   

In [ ]:
df= futuresData[futuresDataKeys[3]][symbolDates[2]]
futuresDataKeys[3]

In [ ]:
dfConcat = pd.concat(list(futuresData[futuresDataKeys[3]].values() ), axis =0, keys = list(futuresData[futuresDataKeys[3]].keys()))

In [ ]:
calcLOBdf = createLOB.calcLOB(createLOB.formatLOB(dfConcat))

In [ ]:
LOBdf =createLOB.formatLOB(calcLOBdf )

In [ ]:
mpChange = LOBdf.MicroPricePctChange.dropna().values

In [ ]:
mpChange.shape

In [ ]:
fig = plt.figure()
ax=plt.hist(mpChange, alpha=0.5, color='r', bins=10)
figuresTarget = os.path.join(figuresFolder, str(futuresDataKeys[3])+"micropriceHisto.png")
plt.title('Microprice for symbol '+ str(futuresDataKeys[3]))
plt.savefig(figuresTarget )
plt.tight_layout()

In [ ]:
from scipy import stats
pd.DataFrame(stats.describe(mpChange))

In [ ]:
from scipy.stats import norm, laplace

def sample_gaussian_vs_laplace(n=220, mu=0.0, sigma2=None, b=np.sqrt(0.5)):    
    # sample from both distributions
    X=norm.rvs(size=n, loc=mu, scale=sigma2)
    Y=laplace.rvs(size=n, loc=mu, scale=b)
    
    return X,Y

In [ ]:
X, Y = sample_gaussian_vs_laplace(n=len(mpChange), mu= np.average(mpChange), sigma2 = np.std(mpChange), b=np.sqrt(0.5))

In [ ]:
plt.hist(X, alpha=0.9)
plt.hist(mpChange, alpha=0.5, color='r');

In [ ]:
samples_p = RealFeatures(X.reshape(1,len(X)))
samples_q = RealFeatures(Y.reshape(1,len(Y)))
samples_z = RealFeatures(mpChange.reshape(1,len(mpChange)))

In [ ]:
mmd = QuadraticTimeMMD(samples_p, samples_z)
kernel = GaussianKernel(10, 1)
mmd.set_kernel(kernel)
alpha = 0.05

In [ ]:
mmd.set_statistic_type(ST_BIASED_FULL)
statistic = mmd.compute_statistic()
mmd.set_null_approximation_method(NAM_PERMUTATION)
mmd.set_num_null_samples(200)
threshold = mmd.compute_threshold(alpha)
p_value = mmd.compute_p_value(statistic)
print(p_value)

In [ ]:
def two_sample_permutation_test(test_statistic, X, Y, num_permutations, prog_bar=True):
    assert X.ndim == Y.ndim
    
    statistics = np.zeros(num_permutations)
    
    range_ = range(num_permutations)
    if prog_bar:
        range_ = tqdm(range_)
    for i in range_:
        # concatenate samples
        if X.ndim == 1:
            Z = np.hstack((X,Y))
        elif X.ndim == 2:
            Z = np.vstack((X,Y))
            
        # IMPLEMENT: permute samples and compute test statistic
        perm_inds = np.random.permutation(len(Z))
        Z = Z[perm_inds]
        X_ = Z[:len(X)]
        Y_ = Z[len(X):]
        my_test_statistic = test_statistic(X_, Y_)
        statistics[i] = my_test_statistic
    return statistics

In [ ]:
import tensorflow as tf
sigma_median= 1

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
#     print("MMD and ratio:", sess.run(mmd.rbf_mmd2_and_ratio(X,Y, sigma=sigma_median)))

In [ ]:
import tqdm 
log_sigmas = np.linspace(-2, 2,10)
                         
ratios = np.empty(len(log_sigmas))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # IMPLEMENT: compute ratio for every sigma
    for i, log_sigma in list(enumerate(log_sigmas)):
        current_ratio = sess.run(mmd.rbf_mmd2_and_ratio(X_learn,Y_learn, sigma=10**log_sigma))[1]
        ratios[i] = current_ratio

plt.plot(log_sigmas, ratios)
plt.grid(True)
plt.xlabel("Log sigma")
plt.ylabel("Ratio of MMD and variance (proportional to test power)")

print("best sigma:", 10**log_sigmas[np.argmax(ratios)])